In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
import pandas as pd
import uuid
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, DataCollatorWithPadding, TrainingArguments, Trainer,pipeline
from datasets import load_dataset, concatenate_datasets

In [ ]:


# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2

import os
import torch
import evaluate
import datasets
import collections
import transformers
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from time import perf_counter
from torch.utils.data import DataLoader
from datasets import (
    load_dataset,
    disable_progress_bar
)
from transformers import (
    pipeline,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    IntervalStrategy
)

device = "cuda" if torch.cuda.is_available() else "cpu"
cache_dir = None

%watermark -a 'Ethen' -d -u -iv

In [ ]:
column_names_qtn=["id", "title", "context", "question", "answers"]
qtndata = []
qtndf=pd.DataFrame(qtndata, columns=column_names_qtn)

In [ ]:
lst={}
lst2={}
lst["what is coffee made of?"]={"text":["roasted coffee beans"], "answer_start":[int(36)]}
lst["what is coffee ingredient?"]={"text":["roasted coffee beans"], "answer_start":[int(36)]}
lst["what are the coffee flavors available?"]={"text":["espresso, french press, caffe latte, or already-brewed canned coffee"], "answer_start":[int(644)]}
lst2["what are the coffee types?"]={"text":["C. arabica and C . robusta. "], "answer_start":[int(51)]}
lst2["what are the coffee varieties?"]={"text":["C. arabica and C . robusta. "], "answer_start":[int(51)]}
lst2["which countries cultivate coffee?"]={"text":["70 countries"], "answer_start":[int(116)]}
context1="coffee is a beverage prepared from roasted coffee beans (ingredient). darkly colored, bitter, (appearance) and slightly acidic, coffee has a stimulating effect on humans, primarily due to its caffeine content. it has the highest sales in the world market for hot drinks. the seeds of the coffea plant fruits are separated to produce unroasted green coffee beans. the beans are roasted and then ground into fine particles typically steeped in hot water before being filtered out, producing a cup of coffee. it is usually served hot, although chilled or iced coffee is common. coffee can be prepared and presented in a variety of ways for e.g., espresso, french press, caffe latte, or already-brewed canned coffee (flavors). sugar, sugar substitutes, milk, and cream are often added to mask the bitter taste or enhance the flavor ."
context2="The two most commonly grown coffee bean types are C. arabica and C . robusta. Coffee plants are cultivated in over 70 countries, primarily in the equatorial regions of the Americas, Southeast Asia, the Indian subcontinent, and Africa. As of 2018, Brazil was the leading grower of coffee beans, producing 35% of the world's total. Green, unroasted coffee is traded as an agricultural commodity. Despite sales of coffee reaching billions of dollars worldwide, farmers producing coffee beans disproportionately live in poverty. Critics of the coffee industry have also pointed to its negative impact on the environment and the clearing of land for coffee-growing and water use."

for key, value in lst.items():
    print(value)
    qtndf.loc[len(qtndf.index)] = [str(uuid.uuid4().hex), 'introduction', context, key, value]
for key, value in lst2.items():
    print(value)
    qtndf.loc[len(qtndf.index)] = [str(uuid.uuid4().hex), 'countries', context2, key, value]

In [ ]:
dataset = load_dataset('squad')

In [ ]:
qatrain=dataset['train'].select(range(100))

In [ ]:
qatest=dataset['validation'].select(range(30))

In [ ]:
df_pandas = qatrain.to_pandas()

In [ ]:
df_pandas.head()

In [ ]:
overall_df = pd.concat([df_pandas, qtndf])
traindataset = Dataset.from_pandas(overall_df)

In [ ]:
qatest

In [ ]:
len(overall_df)

In [ ]:
overall_df.shape

In [ ]:
traindataset = traindataset.remove_columns(["__index_level_0__"])

In [ ]:
traindataset

In [ ]:
dataset

In [ ]:
dataset['train']=traindataset
dataset['validation']=qatest

In [ ]:
dataset

In [ ]:
# experiment with different public model checkpoints
model_checkpoint = "distilbert-base-uncased"
task_name = "squad-try-three" # "squad_v2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, cache_dir=cache_dir)


In [ ]:
# maximum length of a feature (question and context)
max_length = 384
# overlap between two part of the context
doc_stride = 128

In [ ]:
def prepare_qa_train(examples):
    """Prepare training data, input features plus label for question answering dataset."""
    answers = examples["answers"]
    examples["question"] = [question.strip() for question in examples["question"]]
    
    # Tokenize our examples with truncation and padding, but keep overflows using a stride.
    # This results in one example potentially generating several features when a context is
    # long, each of those features having a context that overlaps a bit the previous
    # feature's context to prevent chopping off answer span.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        stride=doc_stride,
        padding="max_length"
    )
    sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
    offset_mapping = tokenized_examples["offset_mapping"]

     # We will label impossible answers with CLS token's index.
    cls_index = 0

    # start_positions and end_positions will be the labels for extractive question answering
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    for i, offset in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]

        sample_index = sample_mapping[i]
        answer = answers[sample_index]
        
        # if no answers are given, set CLS index as answer
        if len(answer["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])

            sequence_ids = tokenized_examples.sequence_ids(i)

            # find the context's corresponding start and end token index
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # if answer is within the context offset, move the token_start_index and token_end_index
            # to two ends of the answer else label it with cls index
            offset_start_char = offset[token_start_index][0]
            offset_end_char = offset[token_end_index][1]
            if offset_start_char <= start_char and offset_end_char >= end_char:
                while token_start_index < len(offset) and offset[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_position = token_start_index - 1

                while offset[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_position = token_end_index + 1

                tokenized_examples["start_positions"].append(start_position)
                tokenized_examples["end_positions"].append(end_position)
            else:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)

    return tokenized_examples

In [ ]:
examples = dataset["train"]
answers = examples["answers"]

tokenized_examples = prepare_qa_train(examples)

start_positions = tokenized_examples["start_positions"]
end_positions = tokenized_examples["end_positions"]
for i, input_ids in enumerate(tokenized_examples["input_ids"]):
    start = start_positions[i]
    end = end_positions[i] + 1
    string = tokenizer.decode(input_ids[start:end])
    print("expected answer:", answers[i]["text"][0])
    print("preprocessing answer:", string)

In [ ]:
# prevents progress bar from flooding our document
disable_progress_bar()

tokenized_datasets = dataset.map(
    prepare_qa_train,
    batched=True,
    remove_columns=dataset["train"].column_names,
    num_proc=8
)
tokenized_datasets

In [ ]:
prepare_qa_train

In [ ]:
model_name = model_checkpoint.split("/")[-1]
fine_tuned_model_checkpoint = f"{model_name}-fine_tuned-{task_name}"

if os.path.isdir(fine_tuned_model_checkpoint):
    do_train = False
    model = AutoModelForQuestionAnswering.from_pretrained(fine_tuned_model_checkpoint, cache_dir=cache_dir)
else:
    do_train = True
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint, cache_dir=cache_dir)

In [ ]:
os.environ['DISABLE_MLFLOW_INTEGRATION'] = 'TRUE'

args = TrainingArguments(
    output_dir=fine_tuned_model_checkpoint,
    learning_rate=0.0001,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=50,
    weight_decay=0.01,
    fp16=False,
    # we set it to evaluate/save per epoch to avoid flowing console
    evaluation_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    load_best_model_at_end=True,
    save_total_limit=2,
    do_train=do_train
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

In [ ]:
if trainer.args.do_train:
    train_output = trainer.train()
    # saving the model which allows us to leverage
    # .from_pretrained(model_path)
    trainer.save_model(fine_tuned_model_checkpoint)

In [ ]:
fine_tuned_model_checkpoint

In [ ]:
qa_pipeline = pipeline(
    "question-answering",
    model=fine_tuned_model_checkpoint,
    tokenizer=fine_tuned_model_checkpoint
)


In [ ]:

output = qa_pipeline({
    "question": "what is coffee made of?",
    "context": context1
})
# answer_text = example["answers"]["text"][0]
# print("output answer matches expected answer: ", output["answer"] == answer_text) 
output

In [ ]:

output = qa_pipeline({
    "question": "what are the coffee types?",
    "context": context2
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)


In [ ]:
qa_dataset

In [ ]:
batch_size = 32
epochs = 2

training_args = TrainingArguments(
    output_dir='./tqanew/results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_dir='./tqanew/logs',
    save_strategy='epoch',
    logging_steps=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
#     report_to="wandb",  # enable logging to W&B
)

trainer = Trainer(
    model=qa_bert,
    args=training_args,
    train_dataset=qa_dataset['train'],
    eval_dataset=qa_dataset['validation'],
    data_collator=data_collator
)

# Get initial metrics
trainer.evaluate()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
http://ethen8181.github.io/machine-learning/deep_learning/question_answer/question_answer.html